### BIDS conversion: creation of desired folder structure, file move and cleanup. 

In [97]:
import os
import json
import datetime

In [118]:
bids_path = "/home/connectomics/Pulpit/mounted_data/BONNA_decide_net/data/main_fmri_study"
nii_path = "/home/connectomics/Pulpit/mounted_data/BONNA_decide_net/data/main_fmri_study_nii"
n_subjects = 33

t1w_name = "Ax_3D_T1FSPGR_BRAVO_new"
task_rew_name = "fmri_prl_rew"
task_pun_name = "fmri_prl_pun"

t1w_bids_suffix = "T1w" 
task_rew_bids_suffix = "task-prlrew_bold"
task_pun_bids_suffix = "task-prlpun_bold"

List all available neuroimaging files. After running look at `img_files` and resolve manually potential conflicts: missing files, duplicates, naming problems. Missing files and duplicates will be detected by the script and printed to the console.

In [42]:
# Store all filenames
img_files = {}

for subject_i in range(1, n_subjects+1):
    
    subject_id = f"m{subject_i:02}"
    subject_files = [file for file in os.listdir(nii_path) if subject_id in file]
    
    if subject_files: 
        
        t1_files = [file for file in subject_files if t1w_name in file]
        fmri_rew_files = [file for file in subject_files if task_rew_name in file]
        fmri_pun_files = [file for file in subject_files if task_pun_name in file]

        # Create dictionary entry
        img_files[subject_id] = {
            't1w_files': t1_files,
            'fmri_rew_files': fmri_rew_files,
            'fmri_pun_files': fmri_pun_files,
            'all_files': subject_files
        }
        
for subject_key in img_files:
    if len(img_files[subject_key]['t1w_files']) != 2:
        print(f"Incorrect number of t1w files for subject {subject_key}")
    if len(img_files[subject_key]['fmri_rew_files']) != 2:
        print(f"Incorrect number of fmri files (reward) for subject {subject_key}")
    if len(img_files[subject_key]['fmri_pun_files']) != 2:
        print(f"Incorrect number of fmri files (punishment) for subject {subject_key}")
        
subjects = list(img_files.keys())

Create basic folder structure

In [37]:
def mkdir_protected(path: str):
    '''Creates directory if it doesn't exist'''
    if not os.path.exists(path):
        os.mkdir(path)
    else:
        print(f'{path} already exists')

In [ ]:
for folder in ['derivatives', 'code', 'sourcedata']:
    mkdir_protected(os.path.join(bids_path, folder))

for subject in subjects:
    
    subject_path = os.path.join(bids_path, f'sub-{subject}')
    img_files[subject]['subject_path'] = subject_path
    mkdir_protected(subject_path)
    
    for folder in ['func', 'anat']:
        
        subject_inner_folder = os.path.join(subject_path, folder) 
        mkdir_protected(subject_inner_folder)
        img_files[subject][f"subject_path_{folder}"] = subject_inner_folder 

Create additional entries to `img_files` dictionary:

- `t1w_bids_name`: proper name of T1 file acording (BIDS compliant)
- ...

In [119]:
for subject in subjects:
    
    # Bids names
    img_files[subject]["t1w_bids_name"] = f"sub-{subject}_{t1w_bids_suffix}"
    img_files[subject]["task_rew_bids_name"] = f"sub-{subject}_{task_rew_bids_suffix}"
    img_files[subject]["task_pun_bids_name"] = f"sub-{subject}_{task_pun_bids_suffix}"
    
    # Original names
    img_files[subject]["t1w_orig_name"] = f"{t1w_name}_{subject}"
    img_files[subject]["task_rew_orig_name"] = f"{task_rew_name}_{subject}"
    img_files[subject]["task_pun_orig_name"] = f"{task_pun_name}_{subject}"

Move and rename imaging sequences.

In [89]:
for subject in subjects:
  
    # T1w file
    for file in img_files[subject]["t1w_files"]:
        
        extension = os.path.splitext(file)[1]
        if extension == ".gz": extension = ".nii.gz"
        
        oldname = os.path.join(
            nii_path, 
            file
        ) 
        newname = os.path.join(
            img_files[subject]["subject_path_anat"], 
            img_files[subject]["t1w_bids_name"] + extension
        ) 
        
        try:    os.rename(oldname, newname)
        except: print(f"file {oldname} not found!")
            
    # EPI files
    for condition in ["rew", "pun"]:
        
        for file in img_files[subject][f"fmri_{condition}_files"]:
            
            extension = os.path.splitext(file)[1]
            if extension == ".gz": extension = ".nii.gz"
                
            oldname = os.path.join(
                nii_path, 
                file
            ) 
            newname = os.path.join(
                img_files[subject]["subject_path_func"], 
                img_files[subject][f"task_{condition}_bids_name"] + extension
            ) 
            
            try:    os.rename(oldname, newname)
            except: print(f"file {oldname} not found!")

Save metadata

In [94]:
def add_meta(path: str, data: dict):
    '''Check if meta exists, and if not saves data as .json file.'''
    if not os.path.exists(path):
        if type(data) not in [dict, str]:
            raise TypeError('data should be either dict or str')   
        with open(path, 'w') as f:
            if type(data) is dict:
                json.dump(data, f, indent= 4)
            if type(data) is str:
                f.write(data)
    else:
        print(f'{path} already exists')

In [121]:
add_meta(os.path.join(bids_path, "derivatives", "img_files"), img_files)

### Add metadata files
    - dataset_description.json
    - README
    - CHANGES
    - task-prlrew_bold.json
    - task-prlpun_bold.json

In [96]:
dataset_description = {
    'Name': 'DecideNet Main fMRI Study',
    'BIDSVersion': '1.2.0',
    'Authors': ['Kamil Bonna', 'Karolina Finc', 'Jaromir Patyk']
}

dataset_description_path = os.path.join(bids_path, 'dataset_description.json')
add_meta(dataset_description_path, dataset_description)

In [99]:
readme = '''# Project

This BIDS folder contains data from main fMRI study in DecideNet project.'''
readme_path = os.path.join(bids_path, 'README')
add_meta(readme_path, readme)

changes = f'''
1.0.0 {str(datetime.date.today())}
    - initial release
'''
changes_path = os.path.join(bids_path, 'CHANGES')
add_meta(changes_path, changes)

In [107]:
for condition in ["rew", "pun"]:

    if condition == "rew":   condition_full = "reward"
    elif condition == "pun": condition_full = "punishment"

    task_dict = {
        "TaskName": f"Probabilistic Reversal Learning ({condition_full} condition)",
        "RepetitionTime": 2,
        "EchoTime": 0.03,
        "InstitutionName": "Nicolaus Copernicus University in Torun"
    }

    task_meta_path = os.path.join(bids_path, f"task-prl{condition}_bold.json")
    add_meta(task_meta_path, task_dict)

### Fix values in .PhaseEncodingDirection field in json sidecar for functional files
    - value "j?" should be changed to "j-"

In [122]:
for subject in subjects:
    for file in os.listdir(os.path.join(bids_path, 'sub-'+subject, 'func')):
        if '.json' in file:
            
            fname_full = os.path.join(bids_path, 'sub-'+subject, 'func', file)
            
            with open(fname_full, 'r') as f:
                data = json.load(f)

            os.remove(fname_full)
            if data['PhaseEncodingDirection'] == 'j?':
                data['PhaseEncodingDirection'] = 'j-' # Apply fix
            
            with open(fname_full, 'w') as f:
                json.dump(data, f, indent= 4)